In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
# from imblearn.ensemble import BalanceRandomForestClassifier

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "symbol", "date", "iexClose"
]

target = ["p_status"]

In [5]:
#  Import and read the .csv.
equities_df = pd.read_csv("resources/cleaned_close1.csv")
equities_df.head(10)

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status
0,PJUL,8/19/2020,27.840,27.710,Investment Trusts/Mutual Funds,27.760,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
1,PIM,8/19/2020,4.260,4.240,Investment Trusts/Mutual Funds,4.230,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
2,PJT,8/19/2020,59.670,58.885,Investment Banks/Brokers,58.600,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
3,PKBK,8/19/2020,13.460,13.460,Regional Banks,12.890,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
4,PICK,8/19/2020,28.490,28.380,Investment Trusts/Mutual Funds,28.250,Miscellaneous,71208,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,Null
5,PK,8/19/2020,8.770,8.605,Real Estate Investment Trusts,8.440,Finance,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
6,PJP,8/19/2020,66.675,66.420,Investment Trusts/Mutual Funds,66.260,Miscellaneous,14548,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
7,PLBC,8/19/2020,20.830,20.895,Regional Banks,20.620,Finance,616,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
8,PKE,8/19/2020,11.480,11.420,Aerospace & Defense,11.290,Electronic Technology,43723,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
9,PJUN,8/19/2020,29.250,28.980,Investment Trusts/Mutual Funds,29.105,Miscellaneous,1365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1


In [6]:
# Load the data
# file_path = Path('equities.csv')
# df = pd.read_csv(file_path, skiprows=1)[:-2]
# df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
equities_df = equities_df.dropna(axis='columns', how='all')

# Drop the null rows
equities_df = equities_df.dropna()
equities_df.drop(equities_df.loc[equities_df['p_status']== "Null"].index, inplace=True)

# # Remove the `Issued` loan status
# issued_mask = df['loan_status'] != 'Issued'
# df = df.loc[issued_mask]

# # convert interest rate to numerical
# df['int_rate'] = df['int_rate'].str.replace('%', '')
# df['int_rate'] = df['int_rate'].astype('float') / 100


# # Convert the target column values to low_risk and high_risk based on their values
# x = {'Current': 'low_risk'}   
# df = df.replace(x)

# x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
# df = df.replace(x)

# df.reset_index(inplace=True, drop=True)

equities_df.head()

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status
0,PJUL,8/19/2020,27.84,27.710,Investment Trusts/Mutual Funds,27.76,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
1,PIM,8/19/2020,4.26,4.240,Investment Trusts/Mutual Funds,4.23,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
2,PJT,8/19/2020,59.67,58.885,Investment Banks/Brokers,58.60,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
3,PKBK,8/19/2020,13.46,13.460,Regional Banks,12.89,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
5,PK,8/19/2020,8.77,8.605,Real Estate Investment Trusts,8.44,Finance,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1


In [8]:
float or np.ndarray(dtype=float)
equities_df['date'] = pd.to_datetime(equities_df['date'])    
equities_df['date_delta'] = (equities_df['date'] - equities_df['date'].min())  / np.timedelta64(1,'D')
# converting to object
equities_df = equities_df.astype({"date":'int64'}) 

In [9]:
equities_df = equities_df.drop(columns=["industry","sector"])
equities_df.head()

,symbol,date,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status,date_delta
0,PJUL,1597795200000000000,27.84,27.710,27.76,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
1,PIM,1597795200000000000,4.26,4.240,4.23,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0,0.0
2,PJT,1597795200000000000,59.67,58.885,58.60,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
3,PKBK,1597795200000000000,13.46,13.460,12.89,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
5,PK,1597795200000000000,8.77,8.605,8.44,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0


# Split the Data into Training and Testing

In [10]:
# Create our features
# X = pd.get_dummies(equities_df, columns=['home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 
#                 'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 
#                 'debt_settlement_flag']).drop('loan_status', axis=1)

# X = pd.get_dummies(equities_df, columns=["symbol", "date"]).drop('change', axis=1)
# X = pd.get_dummies(equities_df, columns=["symbol", "date"]).drop('p_status', axis=1)
X = equities_df.drop(['p_status', "symbol"],axis=1)

# Create target
# y = equities_df.loc[:, target].copy()
y = pd.DataFrame(equities_df["p_status"])

In [11]:
X.describe()

,date,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,date_delta
count,5.704000e+05,570400.000000,570400.000000,570400.000000,5.704000e+05,570400.000000,570400.000000,570400.000000,570400.000000,5.704000e+05,570400.000000,5.704000e+05,570400.000000,5.704000e+05,5.704000e+05,570400.000000
mean,1.603184e+18,42.658513,44.234890,41.554962,9.234301e+05,221402.550000,1275.587500,2559.187500,56305.500000,3.922507e+07,250182.412500,9.540573e+06,99462.837500,1.438905e+08,1.276617e+06,62.375000
std,3.088316e+15,107.594631,108.536875,104.526355,6.664651e+06,38625.202612,801.742567,1341.234424,29470.053439,9.169443e+06,124213.055681,3.574027e+06,68021.723619,4.674591e+07,4.017369e+05,35.744393
min,1.597795e+18,0.000000,0.022900,0.000000,0.000000e+00,165088.000000,246.000000,-752.000000,28849.000000,2.446267e+07,-658774.000000,5.485765e+06,22310.000000,7.635677e+07,5.468590e+05,0.000000
25%,1.600711e+18,8.160000,9.640000,7.805000,1.353775e+04,192707.250000,851.500000,1518.250000,32712.000000,3.177611e+07,212750.750000,6.823470e+06,43793.500000,1.053408e+08,9.472560e+05,33.750000
50%,1.603109e+18,22.340000,23.460000,21.750000,9.192850e+04,212496.500000,1049.000000,2136.000000,41083.000000,3.823387e+07,260824.500000,8.174878e+06,63143.000000,1.356483e+08,1.169490e+06,61.500000
75%,1.605679e+18,43.980000,45.480000,42.865000,4.433702e+05,242672.500000,1371.250000,3486.250000,79775.750000,4.651287e+07,304880.250000,1.158534e+07,161495.250000,1.777278e+08,1.639021e+06,91.250000
max,1.608682e+18,4530.000000,4474.000000,4382.550000,1.864680e+09,317492.000000,3453.000000,5411.000000,119463.000000,5.674314e+07,456078.000000,1.836721e+07,236933.000000,2.412239e+08,2.010951e+06,126.000000


In [12]:
# Check the balance of our target values
y['p_status'].value_counts()

1    314960
0    255440
Name: p_status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
equities_df.dtypes

symbol                       object
date                          int64
high                        float64
iexClose                    float64
low                         float64
volume                        int64
death                         int64
deathIncrease                 int64
hospitalizedIncrease          int64
hospitalizedCurrently         int64
negative                      int64
negativeIncrease              int64
positive                      int64
positiveIncrease              int64
totalTestResults              int64
totalTestResultsIncrease      int64
p_status                     object
date_delta                  float64
dtype: object

In [15]:
equities_df.head()

,symbol,date,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status,date_delta
0,PJUL,1597795200000000000,27.84,27.710,27.76,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
1,PIM,1597795200000000000,4.26,4.240,4.23,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0,0.0
2,PJT,1597795200000000000,59.67,58.885,58.60,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
3,PKBK,1597795200000000000,13.46,13.460,12.89,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
5,PK,1597795200000000000,8.77,8.605,8.44,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0


In [16]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1) 
rf_model.fit(X_train, y_train)
print (rf_model)

RandomForestClassifier(random_state=1)


In [17]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5378362797006989

In [18]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
confusion_matrix(y_test, y_pred)

array([[27812, 36264],
       [28141, 50383]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.43      0.64      0.46      0.53      0.27     64076
          1       0.58      0.64      0.43      0.61      0.53      0.28     78524

avg / total       0.54      0.55      0.53      0.54      0.53      0.28    142600



In [20]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.23307190456539253, 'volume'),
 (0.2230879932958704, 'iexClose'),
 (0.19562199696049715, 'high'),
 (0.19491570258132673, 'low'),
 (0.016392129142063977, 'negativeIncrease'),
 (0.0158748763921075, 'deathIncrease'),
 (0.015340335892262531, 'hospitalizedIncrease'),
 (0.015196197223371132, 'totalTestResultsIncrease'),
 (0.014057170081946357, 'positiveIncrease'),
 (0.012331303825564022, 'hospitalizedCurrently'),
 (0.01084607818387017, 'negative'),
 (0.010727397159329384, 'date'),
 (0.010693402524382221, 'totalTestResults'),
 (0.010643340467350975, 'death'),
 (0.010612880324414138, 'positive'),
 (0.010587291380250871, 'date_delta')]

### Easy Ensemble AdaBoost Classifier

In [21]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
ec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
print (ec_model)

EasyEnsembleClassifier(n_estimators=100, random_state=1)


In [22]:
# Calculated the balanced accuracy score
ec_model.fit(X_train, y_train)
y_pred = ec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5277021123058974

In [23]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
confusion_matrix(y_test, y_pred)

array([[34187, 29889],
       [37545, 40979]], dtype=int64)

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.48      0.53      0.52      0.50      0.53      0.28     64076
          1       0.58      0.52      0.53      0.55      0.53      0.28     78524

avg / total       0.53      0.53      0.53      0.53      0.53      0.28    142600

